# Class Example: Data Literacy with Polars

Professor demonstration of business question translation and grain concepts

## Learning Objectives
1. Understand how business terms map to code operations
2. Learn the concept of "grain" - what each row represents
3. See how `group_by()` changes the grain of your data

In [1]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

Loading diamonds dataset...
Dataset loaded successfully!


## Part 1: Understanding Our Data

Let's first explore what data we have:

In [ ]:
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns}")

In [ ]:
# Display first 5 rows
df.head()

## Part 2: What is the GRAIN of this data?

**Grain** = What each row represents

Let's investigate:

In [ ]:
print(f"Total number of rows: {len(df)}")
print("\nEach row represents: ONE DIAMOND")
print("This is our current grain: diamond-level data")

## Part 3: Business Question Translation

Let's practice translating business questions into code:

### Question 1: What is the TOTAL price of all diamonds?

Keyword: **TOTAL** → `sum()`

In [ ]:
total_price = df.select(pl.col("Price").sum()).item()
print(f"Total price of all diamonds: ${total_price:,.2f}")

### Question 2: What is the AVERAGE carat size?

Keyword: **AVERAGE** → `mean()`

In [ ]:
avg_carat = df.select(pl.col("Carat Weight").mean()).item()
print(f"Average carat size: {avg_carat:.2f} carats")

### Question 3: HOW MANY diamonds do we have?

Keywords: **HOW MANY** → `len()` or `count()`

In [ ]:
count = len(df)
print(f"Number of diamonds: {count}")

### Question 4: What is the HIGHEST price?

Keyword: **HIGHEST** → `max()`

In [ ]:
max_price = df.select(pl.col("Price").max()).item()
print(f"Highest price: ${max_price:,.2f}")

In [ ]:
color_counts = df["Color"].value_counts().sort("count", descending=True)
print("Color frequency (from most to least common):")
color_counts

### Question 9: What is the MOST COMMON color?

Keywords: **MOST COMMON/FREQUENT** → `value_counts()`

In [ ]:
# Count unique values
unique_count = df.select(pl.col("Clarity").n_unique()).item()
print(f"Number of unique clarity grades: {unique_count}")

# Show what they are
unique_values = df.select(pl.col("Clarity").unique().sort()).to_series().to_list()
print(f"The clarity grades are: {unique_values}")

### Question 8: How many UNIQUE/DISTINCT clarity grades exist?

Keywords: **UNIQUE/DISTINCT** → `n_unique()` or `unique()`

In [ ]:
heavy_diamonds = df.filter(pl.col("Carat Weight") >= 2)
print(f"Number of diamonds that weigh at least 2 carats: {len(heavy_diamonds)}")
print("\nFirst 5 heavy diamonds:")
heavy_diamonds.head().select(["Cut", "Color", "Clarity", "Carat Weight", "Price"])

### Question 7: Show diamonds that weigh AT LEAST 2 carats

Keywords: **AT LEAST** → `filter()` with >= condition

In [ ]:
expensive_count = df.filter(pl.col("Price") > 10000).height
total_count = df.height
percentage = (expensive_count / total_count) * 100

print(f"Diamonds over $10,000: {expensive_count}")
print(f"Total diamonds: {total_count}")
print(f"Percentage: {percentage:.2f}%")

### Question 6: What PERCENTAGE of diamonds cost more than $10,000?

Keywords: **PERCENTAGE** → calculation with filter and division

In [ ]:
top_5 = df.sort("Price", descending=True).head(5)
print("Top 5 most expensive diamonds:")
top_5.select(["Cut", "Color", "Clarity", "Carat Weight", "Price"])

## Key Takeaways

1. **Business words map to code operations**
   - "total" → `sum()`
   - "average" → `mean()`
   - "for each" / "by" → `group_by()`
   - "top/bottom N" → `sort().head(n)` / `sort().tail(n)`
   - "percentage" → calculation with filter and division
   - "at least" / "at most" → `filter()` with >= or <=
   - "unique" / "distinct" → `n_unique()` or `unique()`
   - "most common" / "frequent" → `value_counts()`

2. **Grain changes with group_by()**
   - Original grain: one diamond per row
   - After `group_by('Cut')`: one cut type per row
   - After `group_by(['Cut', 'Color'])`: one cut-color combination per row

3. **Always know your grain before aggregating!**

## Part 4: Using GROUP BY - Changing the Grain

When we use `group_by()`, we **change what each row represents**

### Question 5: What is the average price FOR EACH cut quality?

Keywords: 
- **AVERAGE** → `mean()`
- **FOR EACH** → `group_by()`

In [ ]:
print("BEFORE group_by: Grain = one diamond per row")
print(f"Number of rows: {len(df)}")

In [2]:
result = df.group_by("Cut").agg(
    pl.col("Price").mean().alias("avg_price"),
    pl.len().alias("count")
).sort("avg_price", descending=True)

print("AFTER group_by: Grain = one cut quality per row")
print(f"Number of rows: {len(result)}")
result

AFTER group_by: Grain = one cut quality per row
Number of rows: 5


Cut,avg_price,count
str,f64,u32
"""Ideal""",13127.331185,2482
"""Signature-Ideal""",11541.525692,253
"""Very Good""",11484.69687,2428
"""Good""",9326.65678,708
"""Fair""",5886.178295,129


### Question 6: What is the total value of diamonds BY color?

Keywords:
- **TOTAL** → `sum()`
- **BY** → `group_by()`

In [ ]:
result2 = df.group_by("Color").agg(
    pl.col("Price").sum().alias("total_value"),
    pl.len().alias("number_of_diamonds")
).sort("Color")

print("Grain has changed to: one color per row")
result2

## Part 5: Multiple Group By - Complex Grain

We can group by multiple columns, creating more specific grains:

### Question 7: What is the average price FOR EACH cut and color combination?

In [ ]:
result3 = df.group_by(["Cut", "Color"]).agg(
    pl.col("Price").mean().alias("avg_price"),
    pl.len().alias("count")
).sort(["Cut", "Color"])

print("NEW GRAIN: one row per cut-color combination")
print(f"Number of unique combinations: {len(result3)}")
result3.head(10)

## Key Takeaways

1. **Business words map to code operations**
   - "total" → `sum()`
   - "average" → `mean()`
   - "for each" / "by" → `group_by()`

2. **Grain changes with group_by()**
   - Original grain: one diamond per row
   - After `group_by('Cut')`: one cut type per row
   - After `group_by(['Cut', 'Color'])`: one cut-color combination per row

3. **Always know your grain before aggregating!**